# Market Analytics_Campaign Performance Analysis

Objective:

Identify high- and low-performing campaigns

Compare efficiency across channels

Create decision-ready tables for budget optimization

In [1]:
# Step 1: Load Processed Data
import pandas as pd

df = pd.read_csv("../data/processed/ads_consultant_kpi.csv")
df.head()

,Age,Gender,Income,Location,Ad Type,Ad Topic,Ad Placement,Clicks,Click Time,Conversion Rate,CTR,Impressions,Expected_Conversions,Estimated_Cost,Estimated_Revenue,CPA,ROI
0,61.0,Male,35717.43,Urban,Banner,Travel,Social Media,3,2024-01-18 20:45:56.898459,0.0981,0.0737,40.705563,0.2943,1.5,11.772,5.096840,6.848
1,41.0,Male,47453.25,Rural,Video,Travel,Search Engine,5,2023-04-24 20:45:56.898459,0.0937,0.0592,84.459459,0.4685,2.5,18.740,5.336179,6.496
2,49.0,Female,68126.35,Rural,Text,Food,Social Media,4,2024-02-24 20:45:56.898459,0.1912,0.0563,71.047957,0.7648,2.0,30.592,2.615063,14.296
3,68.0,Female,64585.73,Suburban,Text,Health,Website,6,2023-12-13 20:45:56.898459,0.1122,0.0232,258.620690,0.6732,3.0,26.928,4.456328,7.976
4,63.0,Male,21109.40,Urban,Native,Fashion,Search Engine,5,2023-07-02 20:45:56.898459,0.1426,0.0539,92.764378,0.7130,2.5,28.520,3.506311,10.408


In [2]:
# Step 2: Define Campaign Dimensions
CAMPAIGN_COLS = [
    "Ad Type",
    "Ad Placement",
    "Ad Topic",
    "Location"
]

In [3]:
# Step 3: Campaign-Level Aggregation
campaign_perf = (
    df
    .groupby(CAMPAIGN_COLS)
    .agg(
        impressions=("Impressions", "sum"),
        clicks=("Clicks", "sum"),
        expected_conversions=("Expected_Conversions", "sum"),
        cost=("Estimated_Cost", "sum"),
        revenue=("Estimated_Revenue", "sum")
    )
    .reset_index()
)

In [4]:
# Step 4: Recalculate KPIs at Campaign Level
campaign_perf["CTR"] = campaign_perf["clicks"] / campaign_perf["impressions"]
campaign_perf["CVR"] = campaign_perf["expected_conversions"] / campaign_perf["clicks"]
campaign_perf["CPA"] = campaign_perf["cost"] / campaign_perf["expected_conversions"]
campaign_perf["ROI"] = (campaign_perf["revenue"] - campaign_perf["cost"]) / campaign_perf["cost"]

In [5]:
# Step 5: Clean Aggregated Metrics
campaign_perf.replace([float("inf"), -float("inf")], pd.NA, inplace=True)
campaign_perf.dropna(subset=["CTR", "CVR", "CPA", "ROI"], inplace=True)

In [6]:
# Step 6: Filter Campaigns Based on Performance Thresholds
MIN_CLICKS = 100
MIN_CONVERSIONS = 20

campaign_perf = campaign_perf[
    (campaign_perf["clicks"] >= MIN_CLICKS) &
    (campaign_perf["expected_conversions"] >= MIN_CONVERSIONS)
]

In [7]:
# Step 6: Rank Campaigns
campaign_perf["ROI_Rank"] = campaign_perf["ROI"].rank(ascending=False)
campaign_perf["CPA_Rank"] = campaign_perf["CPA"].rank(ascending=True)

campaign_perf.sort_values("ROI_Rank").head(10)

,Ad Type,Ad Placement,Ad Topic,Location,impressions,clicks,expected_conversions,cost,revenue,CTR,CVR,CPA,ROI,ROI_Rank,CPA_Rank
214,Video,Website,Travel,Suburban,7974.882476,186,49.5159,93.0,1980.636,0.023323,0.266215,1.878185,20.297161,1.0,1.0
27,Banner,Social Media,Health,Rural,9390.743515,252,63.3674,126.0,2534.696,0.026835,0.251458,1.988404,19.116635,2.0,2.0
7,Banner,Search Engine,Food,Suburban,4808.850945,189,47.4217,94.5,1896.868,0.039303,0.250908,1.992759,19.072677,3.0,3.0
121,Text,Search Engine,Technology,Suburban,5026.715496,243,60.8562,121.5,2434.248,0.048342,0.250437,1.996510,19.034963,4.0,4.0
124,Text,Search Engine,Travel,Suburban,6432.295885,286,70.5713,143.0,2822.852,0.044463,0.246753,2.026319,18.740224,5.0,5.0
212,Video,Website,Technology,Urban,5114.521354,246,60.5492,123.0,2421.968,0.048098,0.246135,2.031406,18.690797,6.0,6.0
97,Native,Website,Food,Suburban,4488.773333,187,45.6133,93.5,1824.532,0.041659,0.243921,2.049841,18.513711,7.0,7.0
115,Text,Search Engine,Food,Suburban,5941.463926,224,54.4702,112.0,2178.808,0.037701,0.243171,2.056170,18.453643,8.0,8.0
201,Video,Website,Finance,Rural,7232.303102,249,60.4771,124.5,2419.084,0.034429,0.242880,2.058630,18.430394,9.0,9.0
71,Native,Search Engine,Travel,Urban,4990.407984,212,51.3533,106.0,2054.132,0.042481,0.242233,2.064132,18.378604,10.0,10.0


In [8]:
# Step 7: Save Outputs for Dashboard & Report
campaign_perf.to_csv(
    "../data/processed/campaign_performance_summary.csv",
    index=False
)